# MiniFlow Architecture
small tensorFlow clone

## Generic node representation class
Each node might receive input from multiple other nodes.
each node creates a single output, which will likely be passed to other nodes.

Contains two lists:
- one to store references to the inbound ndoes.
- one to store references to the outbound nodes.

Each node will eventually calculate a value that represents its output.
- initialize the value to None to indicate that it exists but hasn't been set yet.

Each node will need to be able to pass values forward and perform backpropagation.
...

In [5]:
class Node(object):
    def __init__(self, inbound_nodes=[]):
        # Properties
        
        # Node(s) from which this Node receives values
        self.inbound_nodes = inbound_nodes
        # Node(s) to which this Node passes values
        self.outbound_nodes = []
        # For each inbound Node here, add this NOde as an outbound Node to _that_ Node.
        for n in self.inbound_nodes:
            n.outbound_nodes.append(self)
        
        # A calculated value
        self.value = None
        
        def forward(self):
            """
            Forward propagation.
            
            Compute the output value vased on `inbound_nodes` and store the result in self.value.
            """
            raise NotImplemented

## Nodes that Calculate - subclasses
While `Node` defines the base set of properties that every node holds, only specialized [subclasses](https://docs.python.org/3/tutorial/classes.html#inheritance) of `Node` will end up in the graph.

The following subclasses of `Node` will be created:
- `Input`
- `Add`

### The `Input` Subclass

In [6]:
class Input(Node):
    def __init__(self):
        # An Input node has no inbound nodes,
        # so no need to pass anything to the Node instantiator.
        Node.__init__(self)
        
    # NOTE: Input node is the only node where the value
    # may be passed as an argument to forward().
    #
    # All other node implementations should get the value
    # of the previous node from self.inbound_nodes
    #
    # Example:
    # val0 = self.inbound_nodes[0].value
    def forawrd(self, value=None):
        # Overwrite the value if one is passed in.
        if value is not None:
            self.value = value

Unlike the other subclasses of `Node`, the `Input` subclass does not actually calculate anything. The `Input` subclass just holds a `value`, such as a data feature or a model parameter (weight/bias).

The `value` can be either set explicitly or with the `forward()` method. This value is then fed through the rest of the neural network.

### The `Add` Subclass

`Add`, which is another subclass of `Node`, actually can perform a calculation (addition).

In [7]:
class Add(Node):
    def __init(self, x, y):
        # Pass the inbound_nodes list [x, y] to 
        # the Node instantiator. The Add subclass takes two
        # inbound nodes, x and y, and adds the values of those nodes.
        Node.__init__(self, [x, y]) 
        
    def forward(self):
        """
        You'll be writing code here in 
        """